In [1]:
from busy_beaver_blaze import Visualizer, BB5_CHAMP, BB6_CONTENDER  # noqa: F401


In [1]:
import itertools
from busy_beaver_blaze import Machine

machine = Machine("1RB0LD_1LC1RD_1LA1LC_1RZ1RE_1RA0RB")
step_limit = 100_000_000

print(f"Created a machine with {machine.program.state_count} states")

# Step 0 is the initial state, so we can run one less than the limit
steps_run = 1 + sum(1 for _ in itertools.islice(machine, step_limit - 1))

print(f"Machine ran for a total: {steps_run:,} steps including initial state")
print(f"Number of ones on tape: {machine.count_ones():,}")

Created a machine with 5 states
Machine ran for a total: 23,554,764 steps including initial state
Number of ones on tape: 4,097


In [1]:
from time import perf_counter
from busy_beaver_blaze import BB5_CHAMP, run_machine_steps

asm_repeat = 50
rust_repeat = 50
python_repeat = 1
step_limit = 100_000_000

def bench(mode, repeats):
    total = 0.0
    steps = None
    nonzeros = None
    for _ in range(repeats):
        start = perf_counter()
        current_steps, current_nonzeros = run_machine_steps(
            BB5_CHAMP,
            step_limit,
            force=mode,
        )
        total += perf_counter() - start
        if steps is None:
            steps = current_steps
            nonzeros = current_nonzeros
        else:
            assert current_steps == steps
            assert current_nonzeros == nonzeros
    return steps, nonzeros, total / repeats, total

asm_steps, asm_nonzeros, asm_avg, asm_total = bench(None, asm_repeat)
rust_steps, rust_nonzeros, rust_avg, rust_total = bench("rust", rust_repeat)
python_steps, python_nonzeros, python_avg, python_total = bench("python", python_repeat)

assert asm_steps == rust_steps == python_steps
assert asm_nonzeros == rust_nonzeros == python_nonzeros

print(
    f"Assembly (auto) ran {asm_steps:,} steps with {asm_nonzeros:,} ones "
    f"(avg {asm_avg:.3f}s per run over {asm_repeat} runs, total {asm_total:.3f}s)"
)
print(
    f"Rust interpreter ran {rust_steps:,} steps with {rust_nonzeros:,} ones "
    f"(avg {rust_avg:.3f}s per run over {rust_repeat} runs, total {rust_total:.3f}s)"
)
print(
    f"Python helper ran {python_steps:,} steps with {python_nonzeros:,} ones "
    f"(avg {python_avg:.3f}s over {python_repeat} run)"
)
print(
    f"Assembly speedup vs Rust: {rust_avg / asm_avg:.2f}x"
)
print(
    f"Rust speedup vs Python: {python_avg / rust_avg:.2f}x"
)



Assembly (auto) ran 47,176,870 steps with 4,098 ones (avg 0.034s per run over 50 runs, total 1.713s)
Rust interpreter ran 47,176,870 steps with 4,098 ones (avg 0.211s per run over 50 runs, total 10.560s)
Python helper ran 47,176,870 steps with 4,098 ones (avg 26.124s over 1 run)
Assembly speedup vs Rust: 6.16x
Rust speedup vs Python: 123.70x
